<a href="https://colab.research.google.com/github/val93s/Machine_learning/blob/main/Copy_of_11_13_2_Activity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Activity 11.13.2

Determining the topics of a text is an interesting—and challenging—problem.  

In this lesson, you'll categorize the topics of about 20,000 posts to 20 different newsgroups. This [20 newsgroups data set](http://qwone.com/~jason/20Newsgroups/) is often used when working with natural langague processing models.

#Step 1: Install the Necessary Packages
* Run the following code block to import the necessary libraries and packages. Make sure to type "y" when asked if you want to proceed.

In [ ]:
#Step 1

import nltk 
nltk.download('stopwords')
!python3 -m spacy download en_core_web_sm
from nltk.corpus import stopwords

# Helpful packages
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
!pip uninstall gensim
! pip install gensim==4.2.0
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
%matplotlib inline




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


2023-03-09 04:46:21.768355: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-09 04:46:23.939872: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-09 04:46:23.939985: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2023-03-09 04:46:23.940011: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

NotImplementedError: ignored

#Step 2: Read in the 20 Newsgroups Data Set
* Run the following code block to read in the 20 Newsgroups data set.
* Although you won't use it in this activity because you're approaching this as an unsupervised learning problem, `target_names` tells you which newsgroup a post came from.
* What is the name of the newsgroup that appears at the top of the data set?


In [ ]:
#Step 2

df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
df.head()

#Step 3: Remove Extra Characters and Symbols
* As you can see in the printed data above, there are a lot of extra characters and symbols like "@" and "\n" in the text that should be removed.
* Run the following code block to remove or replace each symbol or character. Other punctuation will be removed later using `gensim.utils.simple_preprocess`.
* Preview the text of the first newsgroup post. What is the topic of the post?


In [ ]:
#Step 3

# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub(r'\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub(r'\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub(r"\'", "", sent) for sent in data]

pprint(data[:1])

**Step 3 Answer:**



#Step 4: Process the Sentences into Lists of Strings
* Run the following code block to convert the sentences to lists of single strings.  
* `deacc = True` also removes the reamining punctuation.
* Note the format of the first post. It should match the format used for the text of the literary works preprocessed during the word embedding activity.


In [ ]:
#Step 4

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

#Step 5: Remove Stop Words, Lemmatize Words, and Create Bigrams
* Run the following code block to preprocess the data by removing stop words, lemmatizing words, and creating bigrams:


In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
#trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
#trigram_mod = gensim.models.phrases.Phraser(trigram)

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

#def make_trigrams(texts):
#    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# Remove Stop Words
#Define stop words
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])



#Step 6: Tokenize Words and Compute Word Frequencies
* Run the following code block to tokenize the words in the texts and calculate word frequencies.
* How many times does the word "addition" appear in the first newsgroup post?


In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

**Step 6 Answer:**



#Step 7: Model the Topics in the Newsgroup Posts
* Run the following code block to find the topics.
* Most of the hyperparameters have been tuned so that the model will run as quickly and efficiently as possible.
* How many topics will the model find in the data? Hint: Look at the hyperparameter `num_topics`.


In [ ]:
#Step 7

lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

**Step 7 Answer:**



#Step 8: Print the Terms Associated with the Top 10 Topics
* Run the following code block to print the words associated with each topic.
* What are the words associated with the 20th (last) topic?
* How would you label the last topic?


In [ ]:
# Step 8

pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

**Step 8 Answers:**



#Step 9: Visualize the Topics and Explore the Keywords
* Run the following code block to create a visualization of the topics and keywords associated with the topics.
* Hover over each topic on the right side to view the associated topics.
* You can also see how some topics have keywords that overlap (such as topics 1 and 4).
* What is the most relevant keyword for topic 9?


In [ ]:
#Step 9

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

**Step 9 Answer:**

